<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#splitWords" data-toc-modified-id="splitWords-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>splitWords</a></span></li><li><span><a href="#termWeight" data-toc-modified-id="termWeight-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>termWeight</a></span><ul class="toc-item"><li><span><a href="#jieba" data-toc-modified-id="jieba-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>jieba</a></span></li></ul></li><li><span><a href="#parseAttr" data-toc-modified-id="parseAttr-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>parseAttr</a></span></li></ul></div>

In [22]:
import re
import jieba
import jieba.analyse

## splitWords

我可能不需要，这里是将连续单词按照"-""_"以及大小写规则进行分词

In [23]:
re_camel_common = '(^[a-z0-9]+)|([A-Z0-9]+[a-z0-9]*)'
pattern1 = re.compile(re_camel_common)
re_camel = '[A-Z][a-z0-9]+$'
pattern2 = re.compile(re_camel)

def splitWords(words):
    word = words.split("-")
    if len(word) > 1:
        return word

    word = words.split("_")
    if len(word) > 1:
        return word

    # TODO 这里没有细看
    result = []
    ret = pattern1.findall(words)
    for tupl in ret:
        if not tupl[0] == '':
            result.append(tupl[0])
        elif not tupl[1] == '':
            result.append(tupl[1])

            r = pattern2.search(tupl[1])
            if r is None:
                continue

            vt = r.span()
            if vt[0] == 0:
                continue

            v0 = tupl[1][:r.span()[0]]
            v1 = tupl[1][r.span()[0]:]
            if not v0 == '':
                result.append(v0)
            if not v1 == '':
                result.append(v1)

    return result

In [30]:
splitWords("test")

['test']

In [25]:
splitWords("test_test_test")

['test', 'test', 'test']

In [32]:
splitWords("test123TestTest")

['test123', 'Test', 'Test']

## termWeight

In [6]:
def termWeight(attrValue):
    """
    kwd_weight: 保存了word-weight的键值对，不包括idf值过低的词
    qy_seg: 保存了切分后的 word 集合
    topic_wd: 以idf最大的值做为候选结果中的必需值
    """
    if not isinstance(attrValue, str):
        return None, None, None

    # topic_wd以idf最大的值做为候选结果中的必需值，qy_kwd/kwd_weight中保存了word-weight的键值对，不包括idf值过低的词
    kwd_weight = {}
    topic_wd = ''
    qy_kwd = []
    qy_kwd_tmp = jieba.analyse.extract_tags(attrValue, topK=20, withWeight=True, allowPOS=(), withFlag=False)
    for item in qy_kwd_tmp:
        if topic_wd == '':
            topic_wd = item[0]
        subseg = splitWords(item[0])
        if len(subseg) > 1:
            for word in subseg:
                kwd_weight[word] = item[1]
                unit = [word,item[1]]
                qy_kwd.append(unit)
        else:
            kwd_weight[item[0]] = item[1]
            qy_kwd.append([item[0],item[1]])
    
    qy_seg = []
    qy_seg_tmp = jieba.lcut(attrValue)
    # 英文value切词，英文切词是基于jieba的结果
    for item in qy_seg_tmp:
        subseg = splitWords(item)
        if len(subseg) > 1:
            for word in subseg:
                qy_seg.append(word)
        else:
            qy_seg.append(item)
    return kwd_weight, qy_seg, topic_wd

#GET CANDIDATE RESULT


In [34]:
termWeight('This is test test_test')

({'test': 5.97738375145},
 ['This', ' ', 'is', ' ', 'test', ' ', 'test', 'test'],
 'test')

### jieba

In [33]:
attrValue = 'This is test test_test'
jieba.analyse.extract_tags(attrValue,topK=20,withWeight=True,allowPOS=(),withFlag=False)
jieba.lcut(attrValue)

[('test', 5.97738375145), ('test_test', 5.97738375145)]

['This', ' ', 'is', ' ', 'test', ' ', 'test_test']

## parseAttr

In [ ]:
# %load -s parseAttr /tmp/API_RECOM/recomm.py
def parseAttr(attrName, attrValue, wt, candidates, flag_topic):
    if attrName is None or attrValue is None:
        return None
    kwd_weight, qy_seg, topic_wd = termWeight(attrValue)
    if kwd_weight is None or qy_seg is None:
        return None

    #查找倒排索引,将每条候选结果展开成向量
    rst_list = {}
    score_list= {}
    vector_pos = -1
    for kwd_val in qy_seg:
        if not kwd_val in kwd_weight:
            continue
        kwd_tmp = ''
        if attrName == 'NONE':
            kwd_tmp = kwd_val
        else:
            kwd_tmp = attrName + "-" + kwd_val
        kwd = kwd_tmp.lower()
        vector_pos += 1
        if not kwd in invdict:
            continue
        kwd_list = {}
        for one_rst in invdict[kwd]:
            kwd_list[one_rst] = 1
        for one_rst in candidates:
            if one_rst in invdict_all[kwd]:
                kwd_list[one_rst] = 1
        for one_rst in kwd_list:
            if not one_rst in rst_list:
                rst_list[one_rst] = []
            times = vector_pos - len(rst_list[one_rst])
            if times >= 0:
                rst_list[one_rst] = rst_list[one_rst]+[0]*times+[1]
            else:
                rst_list[one_rst][vector_pos] += 1

    #调整每个结果向量的各维度权重
    for a_rst in rst_list:
        #分析kwd之间的连接情况，考虑N-gam(N>2)特征，防止切词粒度太细
        i = 0
        window_start = 0
        window_end = 0
        window_max_weight = -1
        start_is_none = 0
        while i < len(rst_list[a_rst]):
            if not qy_seg[i] in kwd_weight:
                i += 1
                continue
            if rst_list[a_rst][i]>0:
                #调整同一个term多次出现在某行的权值比例
                rst_list[a_rst][i] = 1+(rst_list[a_rst][i]-1)/repeat_weight_punish
                w = kwd_weight[qy_seg[i]]
                #根据invdict_all调整kwd的权值，start
                #kwd_tmp = ''
                #if attrName == 'NONE':
                #    kwd_tmp = qy_seg[i]
                #else:
                #    kwd_tmp = attrName + "-" + qy_seg[i]
                #kwd = kwd_tmp.lower()
                #if kwd in invdict_all:
                #    ln = len(invdict_all[kwd])
                #    if ln > kw_inverted_num_norm:
                #        ln =  kw_inverted_num_norm
                #    if ln < 1:
                #        ln = 1
                    #w = w*(1/ln)
                #end
                if window_max_weight < w:
                    window_max_weight = w
                window_end = i
                if start_is_none > 0:
                    window_start = i
                    start_is_none = 0
            else:
                if window_max_weight > 0:
                    j = window_start
                    while j <= window_end:
                        rst_list[a_rst][j] = window_max_weight
                        j += 1
                start_is_none = 1
                window_start = i
                window_end = window_start
                window_max_weight = -1
            i += 1
        #检查是否最后一个窗口没有执行赋值操作
        if window_max_weight > -1:
            j = window_start
            
            while j <= window_end:
                rst_list[a_rst][j] = window_max_weight
                j += 1
    #主体词要求
    topic_wd_final = ''
    if flag_topic > 0 and not topic_wd is None:
        if attrName == 'NONE':
            topic_wd_tmp = topic_wd
        else:
            topic_wd_tmp = attrName + "-" + topic_wd
        topic_wd_final = topic_wd_tmp.lower()
    #统计得分
    for a_rst in rst_list:
        score = 0.0
        if topic_wd_final == '' or not topic_wd_final in invdict_all:
            for w in rst_list[a_rst]:
                score += w
        else:
            if a_rst in invdict_all[topic_wd_final]:
                for w in rst_list[a_rst]:
                    score += w
        score_list[a_rst] = score*wt
    return score_list

#TRANS FROM API TO VECTOR
